In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")

In [4]:
spacex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                56 non-null     int64  
 1   Flight Number             56 non-null     int64  
 2   Launch Site               56 non-null     object 
 3   class                     56 non-null     int64  
 4   Payload Mass (kg)         56 non-null     float64
 5   Booster Version           56 non-null     object 
 6   Booster Version Category  56 non-null     object 
dtypes: float64(1), int64(3), object(3)
memory usage: 3.2+ KB


In [5]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Recupero los sitios de lanzamientos
spacex_site_df = spacex_df['Launch Site'].unique()

# Create a dash application
app = dash.Dash(__name__)

In [6]:
app.layout = html.Div(children=[html.H1('SpaceX Launch hola Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites (id='site-dropdown')
                                html.Br(),
                                dcc.Dropdown(
                                    options=[
                                        {'label': 'ALL', 'value': 'ALL'},
                                        *[{'label': site, 'value': site} for site in spacex_site_df]
                                    ],
                                    value = 'ALL',
                                    id = 'site-dropdown',
                                    placeholder = 'Select Launch Site ',
                                    searchable = True
                                ),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(min=0, max=10000, step=1000, value=[min_payload, max_payload], marks={i: str(i) for i in range(0, 10001, 1000)}, id='payload-slider'),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
  if entered_site == 'ALL':
      fig = px.pie(
        spacex_df,
        values='class',
        names='Launch Site',
        title=f'Total Successful Launches'
      )
  else:
      # return the outcomes piechart for a selected site
      filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
      class_counts = filtered_df['class'].value_counts().reset_index()
      class_counts.columns = ['class', 'count']

      fig = px.pie(
        class_counts,
        values='count',
        names='class',
        title=f'Total Successful Launches for site {entered_site}'
      )
  return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(site, payload):
  if site == 'ALL':
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    fig = px.scatter(
      filtered_df,
      x='Payload Mass (kg)',
      y='class',
      color='Booster Version Category',
      title='Correlation between Payload and Success for all Sites'
    )
  else:
    filtered_df = spacex_df[(spacex_df['Launch Site'] == site) & (spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    fig = px.scatter(
      filtered_df,
      x='Payload Mass (kg)',
      y='class',
      color='Booster Version Category',
      title='Correlation between Payload and Success for '+ site
    )
  return fig
# Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>